Programación Basica
===

* 60 min | Última modificación: Noviembre 07, 2019

In [1]:
%load_ext bigdata

In [2]:
%pig_start

In [3]:
%timeout 300

### LOAD

El nombre de archivo soporta wilcards; por ejemplo:

    x = LOAD 'files/*.txt';
    
leerá todos los archivos terminados en `.txt`. Por defecto, se espera que los campos en los archivos estén separados por comas.

In [4]:
%%writefile persons.csv
1,Vivian,Hamilton,1971-07-08,green,1
2,Karen,Holcomb,1974-05-23,green,4
3,Cody,Garrett,1973-04-22,orange,1
4,Roth,Fry,1975-01-29,black,1
5,Zoe,Conway,1974-07-03,blue,2
6,Gretchen,Kinney,1974-10-18,viole,1
7,Driscoll,Klein,1970-10-05,blue,5
8,Karyn,Diaz,1969-02-24,red,1
9,Merritt,Guy,1974-10-17,indigo,4
10,Kylan,Sexton,1975-02-28,black,4
11,Jordan,Estes,1969-12-07,indigo,4
12,Hope,Coffey,1973-12-24,green,5
13,Vivian,Crane,1970-08-27,gray,5
14,Clio,Noel,1972-12-12,red,5
15,Hope,Silva,1970-07-01,blue,5
16,Ayanna,Jarvis,1974-02-11,orange,5
17,Chanda,Boyer,1973-04-01,green,4
18,Chadwick,Knight,1973-04-29,yellow,1

Writing persons.csv


In [5]:
%%pig
fs -put persons.csv

 fs -put persons.csv


In [6]:
%%pig
-- 
-- Carga el archivo desde el disco duro
--
u = LOAD 'persons.csv' USING PigStorage(',') 
    AS (id:INT, 
        firstname:CHARARRAY, 
        surname:CHARARRAY, 
        birtday:CHARARRAY, 
        color:CHARARRAY, 
        quantity:INT);
DUMP u;

 -- 
 -- Carga el archivo desde el disco duro
 --
 u = LOAD 'persons.csv' USING PigStorage(',') 
    AS (id:INT, 
        firstname:CHARARRAY, 
        surname:CHARARRAY, 
        birtday:CHARARRAY, 
        color:CHARARRAY, 
        quantity:INT);
 DUMP u;
2022-05-12 21:41:28,019 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:41:28,125 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2022-05-12 21:41:28,128 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2022-05-12 21:41:28,134 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2022-05-12 21:41:29,064 [main] INFO  org.apache.hado

### STORE

In [7]:
%%pig
fs -rmdir output

 fs -rmdir output
rmdir: `output': No such file or directory


In [8]:
%%pig
STORE u INTO 'output';

 STORE u INTO 'output';
2022-05-12 21:41:46,212 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2022-05-12 21:41:46,250 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:41:47,962 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:41:47,973 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 21:41:47,984 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2022-05-12 21:41:48,823 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 21:41:49,257 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: j

In [9]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2022-05-12 21:41 output/_SUCCESS
-rw-r--r--   1 root supergroup        616 2022-05-12 21:41 output/part-m-00000


In [10]:
!hadoop fs -cat output/part-m-00000 | head

1	Vivian	Hamilton	1971-07-08	green	1
2	Karen	Holcomb	1974-05-23	green	4
3	Cody	Garrett	1973-04-22	orange	1
4	Roth	Fry	1975-01-29	black	1
5	Zoe	Conway	1974-07-03	blue	2
6	Gretchen	Kinney	1974-10-18	viole	1
7	Driscoll	Klein	1970-10-05	blue	5
8	Karyn	Diaz	1969-02-24	red	1
9	Merritt	Guy	1974-10-17	indigo	4
10	Kylan	Sexton	1975-02-28	black	4


In [11]:
!hadoop fs -rm output/*  
!hadoop fs -rmdir  output

Deleted output/_SUCCESS
Deleted output/part-m-00000


In [12]:
%%pig
STORE u INTO 'output' USING PigStorage(';');

 STORE u INTO 'output' USING PigStorage(';');
2022-05-12 21:42:07,039 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:42:08,363 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:42:08,375 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 21:42:08,384 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2022-05-12 21:42:08,814 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 21:42:09,236 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652391618376_0003
2022-05-12 21:42:09,240 [JobControl] INFO  org.apache.hadoop.mapred.YARNRunner - Job jar is not present. Not adding any jar to the list of resources.


In [13]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2022-05-12 21:42 output/_SUCCESS
-rw-r--r--   1 root supergroup        616 2022-05-12 21:42 output/part-m-00000


In [14]:
!hadoop fs -cat output/part-m-00000 | head

1;Vivian;Hamilton;1971-07-08;green;1
2;Karen;Holcomb;1974-05-23;green;4
3;Cody;Garrett;1973-04-22;orange;1
4;Roth;Fry;1975-01-29;black;1
5;Zoe;Conway;1974-07-03;blue;2
6;Gretchen;Kinney;1974-10-18;viole;1
7;Driscoll;Klein;1970-10-05;blue;5
8;Karyn;Diaz;1969-02-24;red;1
9;Merritt;Guy;1974-10-17;indigo;4
10;Kylan;Sexton;1975-02-28;black;4


In [15]:
!hadoop fs -rm output/*  
!hadoop fs -rmdir  output

Deleted output/_SUCCESS
Deleted output/part-m-00000


### FOREACH

In [16]:
%%pig
--
-- Los campos del archivo puden ser indicados por nombre 
-- o por posición iniciando en 0
--
v = FOREACH u GENERATE firstname, $2, $3..$5;
DUMP v;

 --
 -- Los campos del archivo puden ser indicados por nombre 
 -- o por posición iniciando en 0
 --
 v = FOREACH u GENERATE firstname, $2, $3..$5;
 DUMP v;
2022-05-12 21:42:26,981 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:42:28,340 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:42:28,351 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 21:42:28,358 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2022-05-12 21:42:28,379 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 21:42:28,799 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652391618376_0004
2022-05-12 21:42:28,803 [JobControl] IN

In [17]:
%%pig
--
-- uso de condicionales 
--
v = FOREACH u GENERATE ($0 > $5 ? 10 : 20) ;
DUMP v;

 --
 -- uso de condicionales 
 --
 v = FOREACH u GENERATE ($0 > $5 ? 10 : 20) ;
 DUMP v;
2022-05-12 21:42:39,610 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:42:40,098 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 21:42:40,106 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 21:42:40,113 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2022-05-12 21:42:40,947 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 21:42:40,970 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652391618376_0005
2022-05-12 21:42:40,973 [JobControl] INFO  org.apache.hadoop.mapred.YARNRunner - Job jar is not present. No

In [ ]:
%%pig
y = FOREACH u GENERATE $1, $3;
DUMP y;

 y = FOREACH u GENERATE $1, $3;


In [ ]:
%%pig
y = FOREACH u GENERATE UPPER($1);
DUMP y;

In [ ]:
%%pig
y = FOREACH u GENERATE [$1, $3];
DUMP y;

### FILTER

In [ ]:
%%pig
y = FILTER u BY $4 MATCHES 'blue';
DUMP y;

### GROUP

In [ ]:
%%pig
y = GROUP u BY $4;
DUMP y;

In [ ]:
%%pig
z = FOREACH y GENERATE u, COUNT(u);
DUMP z;

### DESCRIBE


In [ ]:
%%pig
DESCRIBE u;

### ORDER .. BY ..

In [ ]:
%%pig
y = ORDER u BY $4;
DUMP y;

### DISCTINCT

In [ ]:
%%pig
-- 
-- opera únicamente sobre registros completos
--
y = FOREACH u GENERATE $4;
z = DISCTINCT y;
DUMP z;

### JOIN

In [ ]:
%%writefile jointable.csv
1,A
2,B
3,C
24,X
25,Y
26,Z

In [ ]:
!hadoop fs -put jointable.csv

In [ ]:
%%pig
w = LOAD 'jointable.csv' USING PigStorage(',') 
    AS (id:INT, 
        letter:CHARARRAY);
DUMP w;

In [ ]:
%%pig
w = JOIN u BY id, w BY id;
DUMP w;

### LIMIT

In [ ]:
%%pig
z = LIMIT u 10;
DUMP z;

### SAMPLE

In [ ]:
%%pig
w = SAMPLE u 0.2;
DUMP w;

### CASE

In [ ]:
%%pig
r = FOREACH u GENERATE (
  CASE $0
    WHEN 1 THEN 10
    WHEN 2 THEN 20
    ELSE 3
  END
);
DUMP r;

### Cierre de Apache Pig

In [ ]:
%pig_quit

## Limpieza del sistema

In [ ]:
!hadoop fs -rm *.csv
!rm pig_* *.csv *.pig